# Week 02 — Confidence Interval & Uncertainty (신뢰구간/변동성)

이 노트북은 **실전 통계 → 해석 → 비즈니스 액션**을 훈련하기 위한 템플릿입니다.
각 섹션의 ✅ 체크포인트를 채우면서 진행하세요.


## 1) Business Questions
- 이번 달 성과 변화가 '진짜'인지, 변동성(우연) 범위인지 판단할 수 있는가?
- 국가/채널별 전환율의 불확실성(CI)을 함께 보여줄 수 있는가?

## 2) Why this matters (Theory)
- CI는 '가능한 값의 범위'를 제시해 의사결정에 안정성을 줌
- p-value보다 CI가 '얼마나 좋아졌나'를 직접적으로 보여줌
- 전환율은 Wilson interval이 실무적으로 안정적

## 3) Setup & Data Load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

DATA_PATH = "data/sales_marketing_example_2025.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["date"])

# Quick sanity checks
display(df.head())
print("rows, cols:", df.shape)
print("missing rate (top):")
display(df.isna().mean().sort_values(ascending=False).head(10))

## 4) Helper Functions

In [ ]:
def one_sentence_insight(prompt: str):
    """Write your one-sentence business insight below."""
    print("\n🧾 One-sentence insight (fill this):")
    print(" -", prompt)

def bootstrap_mean_diff(x, y, iters=5000, seed=0):
    """Bootstrap CI for difference in means (x - y)."""
    r = np.random.default_rng(seed)
    x = np.asarray(pd.Series(x).fillna(0))
    y = np.asarray(pd.Series(y).fillna(0))
    diffs = []
    for _ in range(iters):
        xs = r.choice(x, size=len(x), replace=True)
        ys = r.choice(y, size=len(y), replace=True)
        diffs.append(xs.mean() - ys.mean())
    diffs = np.array(diffs)
    return diffs.mean(), np.quantile(diffs, [0.025, 0.975])

def rate_ci_wilson(successes, n, alpha=0.05):
    """Wilson score interval for a proportion."""
    if n == 0:
        return (np.nan, np.nan)
    z = stats.norm.ppf(1 - alpha/2)
    phat = successes / n
    denom = 1 + z**2/n
    center = (phat + z**2/(2*n)) / denom
    half = z * np.sqrt((phat*(1-phat) + z**2/(4*n)) / n) / denom
    return center - half, center + half

---
## 5) Analysis

### 5.1 Daily aggregated metrics + 28-day moving average
✅ 변동성을 줄이기 위해 이동평균을 활용합니다.

In [ ]:
daily = (df.assign(revenue=df['revenue'].fillna(0), spend=df['marketing_spend'].fillna(0))
           .groupby('date')
           .agg(revenue=('revenue','sum'), spend=('spend','sum'), conv=('conversions','sum'), leads=('leads','sum'))
           .sort_index())
daily['rev_ma28'] = daily['revenue'].rolling(28).mean()
daily[['revenue','rev_ma28']].plot(title='Daily revenue & 28-day MA')
plt.show()

In [ ]:
one_sentence_insight('이동평균 관점에서 최근 추세가 상승/하락인지 한 문장')

### 5.2 Bootstrap CI for mean daily revenue
✅ 일 매출 평균의 95% CI를 부트스트랩으로 추정합니다.

In [ ]:
r = daily['revenue'].values
rng = np.random.default_rng(1)
means = []
for _ in range(4000):
    rs = rng.choice(r, size=len(r), replace=True)
    means.append(rs.mean())
means = np.array(means)
print("mean:", means.mean())
print("95% CI:", np.quantile(means, [0.025, 0.975]))

In [ ]:
one_sentence_insight("평균과 CI를 보고 '이번 기간 성과가 확실히 좋아졌다고 말할 수 있는지' 한 문장")

### 5.3 Conversion rate CI (Wilson) by campaign
✅ 전환율은 표본 수에 따라 CI 폭이 달라집니다.

In [ ]:
rows = []
for c, sub in df.groupby('campaign'):
    n = len(sub)
    s = int(sub['conversions'].sum())
    p = s/n
    lo, hi = rate_ci_wilson(s, n)
    rows.append((c, n, p, lo, hi))
ci_df = pd.DataFrame(rows, columns=['campaign','n','conv_rate','ci_low','ci_high']).sort_values('conv_rate', ascending=False)
display(ci_df)

In [ ]:
one_sentence_insight('전환율이 높은 캠페인이라도 CI가 넓으면 어떤 리스크가 있는지 한 문장')

---
## 6) Interpretation Checklist
- 결과가 '유의'하면, **효과 크기/CI**가 실무적으로 의미 있는가?
- 결과가 '비유의'하면, **표본 부족/측정오류/세그먼트 혼합** 가능성은?
- 액션을 한다면, **누구에게/어디에/얼마나** 적용할 것인가?



## Executive Summary (메일/슬라이드에 그대로 붙여넣기)

- **핵심 질문:**  
- **결론(한 문장):**  
- **근거 지표:** (효과크기 / p-value / CI 등)  
- **비즈니스 임팩트:** (예: 월 매출 +X, 마진 -Y 위험)  
- **권장 액션:** (실험/타겟 변경/채널 믹스 조정)  
- **추가 분석 필요:**  
